p()

In [92]:
import numpy as np

In [93]:
PATH_TO_DATA = "naivebayes-21\\trg.csv"

In [94]:
from collections import defaultdict

In [95]:
list_data = []
class_freq = defaultdict(lambda: 0)
corpus = []

with open(PATH_TO_DATA) as csv_file:
    for line in csv_file:
        line = line.replace('\"', '')

        line_lst = line.split(',')
        line_lst[-1] = line_lst[-1].replace('\n', '')

        class_freq[line_lst[1]] += 1 

        corpus.append(line_lst[-1])

        list_data.append(line_lst)


full_csv_data = np.array(list_data)
class_freq.pop('class')


labels = class_freq.keys()
headers = full_csv_data[0]
data = np.asarray(full_csv_data[1:,1:])

In [96]:
print(class_freq.items())
print(labels)

NUM_CLASSES = len(labels)

dict_items([('B', 1602), ('A', 128), ('E', 2144), ('V', 126)])
dict_keys(['B', 'A', 'E', 'V'])


In [97]:
from numpy.random import default_rng

ABSTRT_I = 1
LBL_I = 0

In [98]:
num_instances_for_class = {}

In [99]:
 def get_stratified_kfold_splits(data, k=10):
    instances_of_class = dict()

    # Create a dictionary with key being label, and the value being an array of instances of that label. 
    for i, label in enumerate(labels):
        i_arr = data[:, 0] == label
        instances_of_class[label] = data[i_arr]
        num_instances_for_class[i] = len(instances_of_class[label])
    
    stratified_splits = dict()
    
    # Upsample to divisible by k
    for class_label, class_instances in instances_of_class.items():
        n = len(class_instances)
        upsample_amt = k - (n % k)

        random_indices = np.random.choice(class_instances[:, ABSTRT_I], size=upsample_amt, replace=False)
        random_indices.resize((random_indices.shape[0], 2), refcheck=False)

        random_indices[:, -1] = class_label

        random_indices[:,[0, 1]] = random_indices[:,[1, 0]]

        upsampled_class_instances = np.concatenate((class_instances, random_indices), 0)
        assert len(upsampled_class_instances) % k == 0, "num examples should be divisible by k"

        stratified_splits[class_label] = np.split(upsampled_class_instances, k)

    for i in range(k):
        kth_test_lst = [stratified_splits[label][i] for label in labels]
        kth_train_lst = [stratified_splits[label][j] for label in labels for j in range(k) if j != i]
        
        kth_train_data = np.concatenate(kth_train_lst)
        kth_test_data = np.concatenate(kth_test_lst)
        np.random.shuffle(kth_train_data)
        np.random.shuffle(kth_test_data)

        yield kth_train_data, kth_test_data

In [100]:
stratified_data = get_stratified_kfold_splits(data)
label_ints = {lbl:i for i, lbl in enumerate(labels)}

In [101]:
def get_set_all_words(X):
    all_words = set(word for i in range(len(data)) for word in X[i].split() if word) 

    return all_words

In [102]:
def get_all_words_i(all_words):
    return {word:i for i, word in enumerate(all_words)}

In [103]:
def get_all_words(data):
    X = data[:, ABSTRT_I]
    return get_all_words_i(get_set_all_words(data))

In [104]:
def get_word_counts(data, all_words_i):    
    # ALPHA = 1
    freq_mat = np.ones((len(data), len(all_words_i)+1))

    for i in range(len(data)):
        for word in data[i][ABSTRT_I].split():
            try:
                freq_mat[i,all_words_i[word]] += 1
            except KeyError:
                pass
        freq_mat[i, -1] = label_ints[data[i, LBL_I]]

    return freq_mat

In [105]:
def corr(arr, i):
    ''' Calcualtes the correlation between one column (class) and the rest for the input matrix. Credit to FBruzzesi (https://stackoverflow.com/users/12411536/fbruzzesi)'''
    mean_t = np.mean(arr, axis=0)
    std_t = np.std(arr, axis=0)

    mean_i = mean_t[i]
    std_i = std_t[i]

    mean_xy = np.mean(arr*arr[:,i][:,None], axis=0)

    corr = (mean_xy - mean_i * mean_t)/(std_i * std_t)
    return corr

In [106]:
def select_k_best(freq_mat, k=10):
    R = corr(freq_mat, -1)
    class_correlations = abs(R)

    k_best_i = np.argpartition(class_correlations, -(k+1))[-(k+1):]
    
    # removes self column
    k_best_i = k_best_i[:-1]

    return k_best_i

In [107]:
def convert_to_clf_form(freq_mat, k_best_i):
    X = freq_mat[:, k_best_i]
    y = freq_mat[:, -1]

    return X, y

In [108]:
def get_sum_mat(X, y):
    sum_mat = np.zeros(NUM_CLASSES, X.shape[1])

    for i in range(NUM_CLASSES):
        temp = X[y == i]
        sum_mat[i,:] = np.sum(temp, axis=0)

    return sum_mat, np.sum(sum_mat, axis = 1)

In [109]:
def word_freq_k_best(train, test):
    all_words_i = get_all_words(train)

    freq_mat_train = get_word_counts(train, all_words_i)
    k_best_i = select_k_best(freq_mat_train, NUM_FEATURES)

    freq_mat_test = get_word_counts(test, all_words_i)

    X_train, y_train = convert_to_clf_form(freq_mat_train, k_best_i)
    X_test, y_test = convert_to_clf_form(freq_mat_test, k_best_i)

    return X_train, y_train, X_test, y_test

In [110]:
## TF-IDF
def abstract_to_dict(abstract):
    '''word:count for words in a particular abstract'''
    abstractDict = defaultdict(lambda: 0)
    for word in abstract.split(' '):
        if word:
            abstractDict[word] += 1 
    return abstractDict 

def termFrequency(abstractDict : dict):
    """(# of repetitions of word in a document) / (# of words in a document)"""
    termFrequencies = {}
    numWords = len(abstractDict)
    
    for word, count in abstractDict.items():
        termFrequencies[word] = count/numWords
    return termFrequencies

def get_inverse_document_frequency(abstract_dict_list : list):
    """ used to calculate the weight of rare words across all documents in the corpus
        idf(w) = log(num_docs/freq_word_all_docs)"""
    idf = defaultdict(lambda: 0)
    numAbstracts = len(abstract_dict_list)

    # calculte number of docs containing word
    for abstractDict in abstract_dict_list:
        for word, count in abstractDict.items():
            if count > 0:
                idf[word] += 1 

    from math import log10
    for word, num_docs_containing_word in idf.items():
        idf[word] = log10(numAbstracts/float(num_docs_containing_word))

    return idf

def get_row_tfidf(tf, idf, word_indexes):
    ''' num occurrences of word i in doc j * log(total docs / number of documents containing i) '''
    row = np.zeros((1, len(word_indexes)))
    for word, numOccurences in tf.items():
        word_index = word_indexes[word]
        row[0, word_index] = numOccurences*idf[word] 
    return row

In [111]:
def calculate_tfidf(X):
    abstract_dict_list = []
    term_frequency_lst = []
    all_words = get_set_all_words(X)

    new_X = np.zeros((len(X), len(all_words)))
    word_indexes = {word: i for i, word in enumerate(all_words)}

    for abstract in X:
        abstract_dict = abstract_to_dict(abstract)
        abstract_dict_list.append(abstract_dict)
        term_frequency_lst.append(termFrequency(abstract_dict))

    print("converted abstracts, calculating idf")    
    idf = get_inverse_document_frequency(abstract_dict_list)

    print("calculating tfidf")
    for i,tf in enumerate(term_frequency_lst):
        if (i % 500 == 0):
            print("{}% complete".format(i/len(abstract_dict_list) * 100))
        new_X[i, :] = get_row_tfidf(tf, idf, word_indexes)
    print("100% complete")
    return new_X

In [112]:
X = calculate_tfidf(data[:, ABSTRT_I])

converted abstracts, calculating idf
calculating tfidf
0.0% complete
12.5% complete
25.0% complete
37.5% complete
50.0% complete
62.5% complete
75.0% complete
87.5% complete
100% complete


In [119]:
print(X[X > 0].any())

True


In [121]:
sum_idf = X.sum(axis=0)
k = 20
k_best_i = np.argpartition(sum_idf, -k)[-k:]
print(k_best_i)

[ 7048 24988  7534   426 17796 18526 27190 18407 29270 25025  1918 14526
 21735 12907 10027 11883  5976 19889 29256 19536]


In [123]:
X = X[:, k_best_i]

"https://towardsdatascience.com/comparing-a-variety-of-naive-bayes-classification-algorithms-fc5fa298379e"  
https://geoffruddock.com/naive-bayes-from-scratch-with-numpy/


In [116]:
class NaiveBayes(object):
    """ DIY implementation of binary Naive Bayes classifier based on categorical data"""
    def __init__(self, alpha=1.0):
        """ """
        self.alpha = alpha
        self.prior = None
        self.word_freq = None
        self.word_p = None
        self.num_classes = 0

    def fit(self, X, y):
        n = len(X)

        for i, label in enumerate(labels):
            i_arr = data[:, 0] == label
            instances_of_class[label] = data[i_arr]
            self.num_classes += 1

        self.prior = np.array([len(instances_of_class[label]) / n for label in instances_of_class])

        self.word_freq = np.array([class_arr.sum(axis=0) for class_arr in instances_of_class]) + self.alpha

        self.lk_word = self.word_freq / self.word_freq.sum(axis=1).reshape(-1, 1)

        return self

    def predict_proba(self, X):
        """ Predict probability of class membership """

        # loop over each observation to calculate conditional probabilities
        class_numerators = np.zeros((len(X), self.num_classes))
        for i, x in enumerate(X):
            word_exists = x.astype(bool)
            lk_words_present = self.lk_word[:, word_exists] ** x[word_exists]
            lk_message = (lk_words_present).prod(axis=1)
            class_numerators[i] = lk_message * self.prior

        normalize_term = class_numerators.sum(axis=1).reshape(-1, 1)
        conditional_probas = class_numerators / normalize_term
        return conditional_probas

    def predict(self, X: matrix_like):
        """ Predict class with highest probability """
        return self.predict_proba(X).argmax(axis=1)

NameError: name 'matrix_like' is not defined

In [170]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

classifiers = [GaussianNB(), BernoulliNB(), CategoricalNB(), ComplementNB(), MultinomialNB()]
classifiers_names = ["Gaussian", "Bernoulli", "Categorical", "complement", "Multinomial"]

arr = np.zeros((5, 11))
count = 0

while True:
    try:
        tr, te = next(stratified_data)

        X_train, y_train, X_test, y_test = train_clf(tr,te)

        for i,clf in enumerate(classifiers):
            clf.fit(X_train, y_train)
            print(clf.score(X_test, y_test))
        count += 1
    except StopIteration:
        break